In [2]:
import os
import json

# 设置你的 imagesTr 和 labelsTr 路径
imagesTr_dir = "/yuming/CoTr/data/nnUNet_raw_data_base/nnUNet_raw_data/Task071_AbdomenCT/imagesTr"
labelsTr_dir = "/yuming/CoTr/data/nnUNet_raw_data_base/nnUNet_raw_data/Task071_AbdomenCT/labelsTr"
# imagesTs_dir = "/yuming/CoTr/data/nnUNet_raw_data_base/nnUNet_raw_data/Task072_AbdomenMR/imagesTs"

# 读取 imagesTr 目录下所有 .nii.gz 文件
images = sorted([f for f in os.listdir(imagesTr_dir) if f.endswith(".nii.gz")])

# 自动构建 training 列表
training = []
for img in images:
    # 去掉最后一个 _0000
    img_base = img.replace("_0000", "").replace(".nii.gz", "")  # 去掉_0000 和 后缀
    img_path = "./imagesTr/" + img_base + ".nii.gz"
    label_path = "./labelsTr/" + img_base + ".nii.gz"
    training.append({"image": img_path, "label": label_path})

# # 2. 构建 test 列表
# test_images = sorted([f for f in os.listdir(imagesTs_dir) if f.endswith(".nii.gz")])
# test = []
# for img in test_images:
#     img_base = img.replace("_0000", "").replace(".nii.gz", "")  # 去掉_0000 和 后缀
#     test.append(f"./imagesTs/{img_base}.nii.gz")

# 创建最终的 dataset.json
dataset_json = {
    "name": "Dataset701_FLARE2022",
    "description": "This dataset was from MICCAI AMOS 2022 Challenge...",
    "tensorImageSize": "3D",
    "reference": "MICCAI FLARE 2022",
    "licence": "CC BY-SA 4.0",
    "release": "0.0",
    "modality": {
        "0": "CT"
    },
    "labels": {
        "0": "background",
        "1": "liver",
        "2": "right kidney",
        "3": "spleen",
        "4": "pancreas",
        "5": "aorta",
        "6": "inferior vena cava",
        "7": "right adrenal gland",
        "8": "left adrenal gland",
        "9": "gallbladder",
        "10": "esophagus",
        "11": "stomach",
        "12": "duodenum",
        "13": "left kidney"
    },
    "numTraining": len(training),
    "numTest": 0,
    "training": training,
    "test": []
}

# 保存到 dataset.json
output_path = "/yuming/CoTr/data/nnUNet_raw_data_base/nnUNet_raw_data/Task071_AbdomenCT/dataset.json"
with open(output_path, "w") as f:
    json.dump(dataset_json, f, indent=4)

print(f"dataset.json saved to {output_path}")


dataset.json saved to /yuming/CoTr/data/nnUNet_raw_data_base/nnUNet_raw_data/Task071_AbdomenCT/dataset.json


In [5]:
import pickle
with open('/yuming/CoTr/data/nnUNet_preprocessed/Task071_AbdomenCT/splits_final.pkl', 'rb') as f:
    plans_amos = pickle.load(f)
# with open('/yuming/CoTr/data/splits_final.pkl', 'rb') as f:
#     plans_BCV = pickle.load(f)
# with open('/yuming/CoTr/plans_my_amos.pkl', 'rb') as f:
#     plans_my_amos = pickle.load(f)
# 在 plans 里查找 fold 列表，通常叫做 plans['splits'] 或者 plans['folds']
# print(plans_amos)
print(type(plans_amos))           # <class 'list'>
print(type(plans_amos[0]))        # <class 'collections.OrderedDict'>
print(plans_amos[0].keys())       # odict_keys(['train', 'val'])
print(plans_amos)       # odict_keys(['train', 'val'])

# print(plans.get('splits') or plans.get('folds'))

<class 'list'>
<class 'collections.OrderedDict'>
odict_keys(['train', 'val'])
[OrderedDict([('train', ['amos_0507', 'amos_0508', 'amos_0510', 'amos_0514', 'amos_0517', 'amos_0518', 'amos_0522', 'amos_0530', 'amos_0532', 'amos_0538', 'amos_0540', 'amos_0541', 'amos_0544', 'amos_0545', 'amos_0546', 'amos_0547', 'amos_0548', 'amos_0549', 'amos_0550', 'amos_0551', 'amos_0552', 'amos_0553', 'amos_0554', 'amos_0555', 'amos_0556', 'amos_0557', 'amos_0558', 'amos_0559', 'amos_0561', 'amos_0562', 'amos_0563', 'amos_0568', 'amos_0570', 'amos_0571', 'amos_0572', 'amos_0573', 'amos_0575', 'amos_0576', 'amos_0578', 'amos_0580', 'amos_0581', 'amos_0582', 'amos_0583', 'amos_0584', 'amos_0585', 'amos_0586', 'amos_0587', 'amos_0588', 'amos_0589', 'amos_0590', 'amos_0591', 'amos_0592', 'amos_0593', 'amos_0594', 'amos_0595', 'amos_0596', 'amos_0597', 'amos_0598', 'amos_0599', 'amos_0600']), ('val', ['amos_7016', 'amos_7164', 'amos_7236', 'amos_7237', 'amos_7240', 'amos_7244', 'amos_7261', 'amos_7264', 'a

In [3]:
import os
import pickle
from collections import OrderedDict

# —— 配置区 —— #
train_dir = "/yuming/Dataset702_AbdomenMR/imagesTr"  # 改为你的训练数据文件夹
val_dir   = "/yuming/Dataset702_AbdomenMR/labelsTs"    # 改为你的验证/测试数据文件夹
output_path = "/yuming/CoTr/dataset/mr/splits_final.pkl"             # 输出文件名
# ———————— #

def extract_id(fname):
    """
    针对训练图像：去掉 _0000.nii.gz，保留 ID
    针对标签文件：去掉 .nii.gz，保留 ID
    """
    base = os.path.basename(fname)
    
    if base.endswith("_0000.nii.gz"):
        return base.replace("_0000.nii.gz", "")
    elif base.endswith(".nii.gz"):
        return base.replace(".nii.gz", "")
    else:
        return base  # fallback

def collect_ids(folder):
    ids = set()
    for fn in os.listdir(folder):
        if fn.lower().endswith(('.nii.gz', '.nii')):
            ids.add(extract_id(fn))
    return sorted(ids)

# 收集
train_ids = collect_ids(train_dir)
print("train_ids", train_ids)
val_ids   = collect_ids(val_dir)

# 构造 OrderedDict 列表
plans = [
    OrderedDict([
        ("train", train_ids),
        ("val",   val_ids),
    ])
]

# 序列化到 pkl
with open(output_path, 'wb') as f:
    pickle.dump(plans, f)

print(f"✔ 已生成 {output_path}")
print(f"  train: {len(train_ids)} 条 ID，示例：{train_ids[:5]}")
print(f"  val:   {len(val_ids)} 条 ID，示例：{val_ids[:5]}")

train_ids ['amos_0507', 'amos_0508', 'amos_0510', 'amos_0514', 'amos_0517', 'amos_0518', 'amos_0522', 'amos_0530', 'amos_0532', 'amos_0538', 'amos_0540', 'amos_0541', 'amos_0544', 'amos_0545', 'amos_0546', 'amos_0547', 'amos_0548', 'amos_0549', 'amos_0550', 'amos_0551', 'amos_0552', 'amos_0553', 'amos_0554', 'amos_0555', 'amos_0556', 'amos_0557', 'amos_0558', 'amos_0559', 'amos_0561', 'amos_0562', 'amos_0563', 'amos_0568', 'amos_0570', 'amos_0571', 'amos_0572', 'amos_0573', 'amos_0575', 'amos_0576', 'amos_0578', 'amos_0580', 'amos_0581', 'amos_0582', 'amos_0583', 'amos_0584', 'amos_0585', 'amos_0586', 'amos_0587', 'amos_0588', 'amos_0589', 'amos_0590', 'amos_0591', 'amos_0592', 'amos_0593', 'amos_0594', 'amos_0595', 'amos_0596', 'amos_0597', 'amos_0598', 'amos_0599', 'amos_0600']
✔ 已生成 /yuming/CoTr/dataset/mr/splits_final.pkl
  train: 60 条 ID，示例：['amos_0507', 'amos_0508', 'amos_0510', 'amos_0514', 'amos_0517']
  val:   50 条 ID，示例：['amos_7016', 'amos_7164', 'amos_7236', 'amos_7237', 'am